In [1]:
import gym
import matplotlib.pyplot as plt
from IPython import display

from krazy_gridworld import KrazyGridWorld
import numpy as np

In [2]:
def show_state(env, step=0, info=""):
    plt.figure(3)
    plt.clf()
    plt.imshow(env.get_img_pyplot_obs())
    plt.title("%s | Step: %d %s" % (env, step, info))
    plt.axis('off')

    display.clear_output(wait=True)
    display.display(plt.gcf())

In [3]:
env = KrazyGridWorld(9)